---

## 🌐 Phase 1: Web Scraping

### 1.1 Baseball Reference Scraper (`baseball_scraper.py`)

**Source:** [Baseball Reference](https://www.baseball-reference.com)

**Technology Stack:**
- **Selenium WebDriver** - For browser automation (handles JavaScript-rendered content)
- **BeautifulSoup** - For HTML parsing
- **pandas** - For DataFrame creation and CSV export
- **webdriver-manager** - Automatic ChromeDriver management

**Key Features:**
- Headless browser mode for faster scraping
- Handles hidden tables (Baseball Reference stores some tables in HTML comments)
- Timeout management to prevent hanging on slow pages
- Anti-detection measures (custom user agent, disabled automation flags)

**Data Collected:**

| File | Description | Key Statistics |
|------|-------------|----------------|
| `Batting_YEAR.csv` | Team batting stats | BA, OBP, SLG, OPS, HR, RBI, R, etc. |
| `Pitching_YEAR.csv` | Team pitching stats | ERA, WHIP, SO, BB, W-L%, etc. |
| `Fielding_YEAR.csv` | Team fielding stats | Fld%, E, DP, DefEff, etc. |
| `Postseason_YEAR.csv` | Playoff results | Series winners/losers, scores |
| `WAA_Positions_YEAR.csv` | Wins Above Average by position | Position-specific WAA rankings |

### 1.2 Salary Scraper (`salary_scraper.py`)

**Source:** [SteveTheUmp.com](https://www.stevetheump.com/Payrolls.htm)

**Purpose:** Scrape historical team payroll data to analyze the relationship between team spending and performance.

**Key Features:**
- Parses multiple tables from a single page (one per year)
- Uses regex to identify year sections from headers
- Extracts team names and payroll amounts

**Data Collected:**

| File | Description | Key Statistics |
|------|-------------|----------------|
| `Salaries_YEAR.csv` | Team payroll data | Team name, Total payroll ($) |

---

## 🧹 Phase 2: Data Cleaning

### 2.1 Team Name Standardization (`data_cleaning.py`)

**Problem:** Team names and abbreviations vary across years and data sources due to:
- Team relocations (Montreal Expos → Washington Nationals)
- Team renamings (Cleveland Indians → Cleveland Guardians)
- Inconsistent abbreviations (CWS vs CHW for White Sox)

**Solution:** All team names are standardized to **2025 conventions**.

#### Historical Name Changes Handled:

| Old Name | New Name (2025 Standard) | Year Changed |
|----------|--------------------------|---------------|
| Montreal Expos | Washington Nationals | 2005 |
| Florida Marlins | Miami Marlins | 2012 |
| Cleveland Indians | Cleveland Guardians | 2022 |
| Oakland Athletics | Athletics | 2024 |
| Tampa Bay Devil Rays | Tampa Bay Rays | 2008 |
| Anaheim Angels | Los Angeles Angels | Various |
| California Angels | Los Angeles Angels | Various |

#### Abbreviation Mappings:

| Old Abbreviation | New (2025 Standard) |
|------------------|---------------------|
| OAK | ATH |
| CWS | CHW |
| FLA | MIA |
| MON | WSN |
| ANA/CAL | LAA |
| TBD | TBR |

### 2.2 Salary Data Cleaning (`salary_cleaning.py`)

**Tasks Performed:**
1. **Identify correct payroll column** - Some source tables have multiple numeric columns
2. **Remove aggregate rows** - Filter out average/median salary rows
3. **Clean currency formatting** - Remove `$`, commas, and `M` suffixes
4. **Standardize team names** - Apply same mappings as main data cleaning

**Before Cleaning:**
```
Oakland Athletics, $45,500,000
```

**After Cleaning:**
```
Athletics, 45500000
```

---

## 📊 Data Overview

Let's examine the structure and sample data from each file type.

In [ ]:
import pandas as pd
import os
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Base data path
DATA_PATH = Path('Data')

# Example year to display
SAMPLE_YEAR = 2025

### Batting Statistics

In [3]:
# Load batting data
batting = pd.read_csv(DATA_PATH / str(SAMPLE_YEAR) / f'Batting_{SAMPLE_YEAR}.csv')
print(f"Batting Statistics ({SAMPLE_YEAR})")
print(f"Shape: {batting.shape[0]} teams × {batting.shape[1]} statistics\n")
print("Columns:", list(batting.columns))
batting.head()

Batting Statistics (1999)
Shape: 31 teams × 29 statistics

Columns: ['Tm', '#Bat', 'BatAge', 'R/G', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB', 'LOB']


,Tm,#Bat,BatAge,R/G,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,LOB
0,Anaheim Angels,45,28.6,4.39,162,6132,5494,711,1404,248,22,158,673,71,45,511,1022,0.256,0.322,0.395,0.716,83,2170,135,43,41,42,24,1097
1,Arizona Diamondbacks,43,30.0,5.60,162,6415,5658,908,1566,289,46,216,865,137,39,588,1045,0.277,0.347,0.459,0.805,101,2595,94,48,61,60,52,1169
2,Atlanta Braves,44,29.6,5.19,162,6351,5569,840,1481,309,23,197,791,148,66,608,962,0.266,0.341,0.436,0.777,96,2427,120,53,74,47,62,1155
3,Baltimore Orioles,43,32.5,5.25,162,6409,5637,851,1572,299,21,203,804,107,46,615,890,0.279,0.353,0.447,0.800,108,2522,146,61,41,55,34,1241
4,Boston Red Sox,48,28.8,5.16,162,6321,5579,836,1551,334,42,176,808,67,39,597,928,0.278,0.350,0.448,0.798,99,2497,131,55,34,56,27,1213


### Pitching Statistics

In [4]:
# Load pitching data
pitching = pd.read_csv(DATA_PATH / str(SAMPLE_YEAR) / f'Pitching_{SAMPLE_YEAR}.csv')
print(f"Pitching Statistics ({SAMPLE_YEAR})")
print(f"Shape: {pitching.shape[0]} teams × {pitching.shape[1]} statistics\n")
print("Columns:", list(pitching.columns))
pitching.head()

Pitching Statistics (1999)
Shape: 31 teams × 36 statistics

Columns: ['Tm', '#P', 'PAge', 'RA/G', 'W', 'L', 'W-L%', 'ERA', 'G', 'GS', 'GF', 'CG', 'tSho', 'cSho', 'SV', 'IP', 'H', 'R', 'ER', 'HR', 'BB', 'IBB', 'SO', 'HBP', 'BK', 'WP', 'BF', 'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'LOB']


,Tm,#P,PAge,RA/G,W,L,W-L%,ERA,G,GS,GF,CG,tSho,cSho,SV,IP,H,R,ER,HR,BB,IBB,SO,HBP,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,LOB
0,Anaheim Angels,20,31.6,5.10,70,92,0.432,4.79,162,162,158,4,7,0,37,1431.1,1472,826,762,177,624,17,877,56,5,65,6258,101,4.94,1.464,9.3,1.1,3.9,5.5,1.41,1138
1,Arizona Diamondbacks,20,30.6,4.17,100,62,0.617,3.77,162,162,146,16,9,4,42,1467.1,1387,676,615,176,543,48,1198,49,10,39,6233,122,4.27,1.315,8.5,1.1,3.3,7.3,2.21,1155
2,Atlanta Braves,22,28.6,4.08,103,59,0.636,3.63,162,162,153,9,9,1,45,1471.0,1398,661,593,142,507,55,1197,26,3,34,6218,123,3.85,1.295,8.6,0.9,3.1,7.3,2.36,1144
3,Baltimore Orioles,21,30.1,5.03,78,84,0.481,4.77,162,162,145,17,11,4,33,1435.0,1468,815,760,198,647,34,982,49,6,55,6259,97,5.01,1.474,9.2,1.2,4.1,6.2,1.52,1139
4,Boston Red Sox,25,30.1,4.43,94,68,0.580,4.00,162,162,156,6,12,1,50,1436.2,1396,718,638,160,469,25,1131,55,0,28,6120,126,4.10,1.298,8.7,1.0,2.9,7.1,2.41,1092


### Fielding Statistics

In [5]:
# Load fielding data
fielding = pd.read_csv(DATA_PATH / str(SAMPLE_YEAR) / f'Fielding_{SAMPLE_YEAR}.csv')
print(f"Fielding Statistics ({SAMPLE_YEAR})")
print(f"Shape: {fielding.shape[0]} teams × {fielding.shape[1]} statistics\n")
print("Columns:", list(fielding.columns))
fielding.head()

Fielding Statistics (1999)
Shape: 31 teams × 16 statistics

Columns: ['Tm', '#Fld', 'RA/G', 'DefEff', 'G', 'GS', 'CG', 'Inn', 'Ch', 'PO', 'A', 'E', 'DP', 'Fld%', 'Rtot', 'Rtot/yr']


,Tm,#Fld,RA/G,DefEff,G,GS,CG,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr
0,Anaheim Angels,45,5.10,0.699,162,1458,1155,12882.0,6123,4294,1723,106,156,0.983,72,7
1,Arizona Diamondbacks,42,4.17,0.701,162,1458,1135,13206.0,6096,4402,1590,104,132,0.983,42,4
2,Atlanta Braves,44,4.08,0.694,162,1458,1021,13239.0,6182,4413,1658,111,127,0.982,43,4
3,Baltimore Orioles,42,5.03,0.697,162,1458,1128,12915.0,6175,4305,1781,89,191,0.986,39,4
4,Boston Red Sox,48,4.43,0.693,162,1458,1188,12930.0,5985,4310,1548,127,132,0.979,48,4


### Salary Data

In [6]:
# Load salary data
salaries = pd.read_csv(DATA_PATH / str(SAMPLE_YEAR) / f'Salaries_{SAMPLE_YEAR}.csv')
print(f"Team Salaries ({SAMPLE_YEAR})")
print(f"Shape: {salaries.shape[0]} teams\n")
salaries.head(10)

Team Salaries (1999)
Shape: 30 teams



,Tm,Payroll
0,New York Yankees,88180712
1,Texas Rangers,81576598
2,Atlanta Braves,74890000
3,Cleveland Guardians,73278458
4,Baltimore Orioles,72198363
5,Boston Red Sox,71725000
6,New York Mets,71506427
7,Los Angeles Dodgers,71115786
8,Arizona Diamondbacks,70496000
9,Chicago Cubs,55443500


### Postseason Results

In [7]:
# Load postseason data
postseason = pd.read_csv(DATA_PATH / str(SAMPLE_YEAR) / f'Postseason_{SAMPLE_YEAR}.csv')
print(f"Postseason Results ({SAMPLE_YEAR})")
postseason

Postseason Results (1999)


,0,1,2
0,World Series,4-0,New York Yankees over Atlanta Braves
1,ALCS,4-1,New York Yankees over Boston Red Sox
2,NLCS,4-2,Atlanta Braves over New York Mets
3,AL Division Series,3-2,Boston Red Sox over Cleveland Indians
4,AL Division Series,3-0,New York Yankees over Texas Rangers
5,NL Division Series,3-1,Atlanta Braves over Houston Astros
6,NL Division Series,3-1,New York Mets over Arizona Diamondbacks


### WAA by Position

In [8]:
# Load WAA positions data
waa = pd.read_csv(DATA_PATH / str(SAMPLE_YEAR) / f'WAA_Positions_{SAMPLE_YEAR}.csv')
print(f"Wins Above Average by Position ({SAMPLE_YEAR})")
print(f"Shape: {waa.shape[0]} teams × {waa.shape[1]} positions\n")
print("Columns:", list(waa.columns))
waa.head()

Wins Above Average by Position (1999)
Shape: 30 teams × 17 positions

Columns: ['Rk', 'Total', 'All P', 'SP', 'RP', 'Non-P', 'C', '1B', '2B', '3B', 'SS', 'LF', 'CF', 'RF', 'OF (All)', 'DH', 'PH']


,Rk,Total,All P,SP,RP,Non-P,C,1B,2B,3B,SS,LF,CF,RF,OF (All),DH,PH
0,1,Arizona Diamondbacks19.9,ATL12.2,ARI9.1,TEX4.5,CLE12.7,TEX4.2,HOU5.1,CLE4.3,ATL4.8,NYY5.5,ARI4.3,ATL5.0,CLE5.0,CLE8.1,SEA2.5,BAL0.1
1,2,Atlanta Braves17.7,ARI10.6,BOS8.8,NYM4.0,CIN11.9,NYM2.5,PIT3.4,NYM3.9,NYM4.4,BOS4.2,MIL3.1,HOU3.7,PHI3.9,KCR7.7,OAK2.5,ARI0.1
2,3,New York Yankees15.3,NYY10.2,ATL8.2,NYY3.6,BAL10.7,DET1.4,OAK3.3,HOU2.8,PHI2.8,CLE3.3,KCR2.9,CIN3.7,TOR3.8,PHI6.3,TEX2.4,SFG0.0
3,4,New York Mets14.7,HOU10.0,HOU7.9,COL3.5,NYM9.6,TBD0.7,NYM3.2,ARI2.7,MIL2.5,TOR3.2,BAL2.6,CLE3.2,CHW3.3,ATL6.2,BAL1.5,CIN-0.2
4,5,Houston Astros14.6,BOS9.9,SEA7.1,ATL3.0,ARI9.3,SFG0.7,STL2.8,SFG1.9,LAD2.0,CIN3.1,CIN1.8,NYY2.9,CHC2.4,CIN6.2,TBD0.7,OAK-0.2


---

## 📈 Data Coverage Summary

In [9]:
# Check data availability across all years
years = range(1998, 2026)
file_types = ['Batting', 'Pitching', 'Fielding', 'Postseason', 'Salaries', 'WAA_Positions']

coverage = []
for year in years:
    year_path = DATA_PATH / str(year)
    if year_path.exists():
        row = {'Year': year}
        for ft in file_types:
            file_path = year_path / f'{ft}_{year}.csv'
            row[ft] = '✓' if file_path.exists() else '✗'
        coverage.append(row)

coverage_df = pd.DataFrame(coverage)
print("Data Coverage by Year and File Type:")
print(f"Years covered: {coverage_df['Year'].min()} - {coverage_df['Year'].max()}")
print(f"Total years: {len(coverage_df)}\n")
coverage_df

Data Coverage by Year and File Type:
Years covered: 1998 - 2025
Total years: 28



,Year,Batting,Pitching,Fielding,Postseason,Salaries,WAA_Positions
0,1998,✓,✓,✓,✓,✓,✓
1,1999,✓,✓,✓,✓,✓,✓
2,2000,✓,✓,✓,✓,✓,✓
3,2001,✓,✓,✓,✓,✓,✓
4,2002,✓,✓,✓,✓,✓,✓
5,2003,✓,✓,✓,✓,✓,✓
6,2004,✓,✓,✓,✓,✓,✓
7,2005,✓,✓,✓,✓,✓,✓
8,2006,✓,✓,✓,✓,✓,✓
9,2007,✓,✓,✓,✓,✓,✓


---

## 🧹 Phase 3: Data Cleaning & Standardization

Now we'll clean and standardize all the data to prepare for analysis.

### 3.1 Team Name Standardization Mappings

First, let's define our standardization mappings to ensure all team names are consistent across all years (using 2025 conventions).

In [10]:
# Team name standardization mappings (Historical names -> 2025 Standard)
TEAM_NAME_MAPPINGS = {
    # Oakland Athletics -> Athletics (2024 relocation)
    "Oakland Athletics": "Athletics",
    "Oakland A's": "Athletics",
    
    # Cleveland name change (2022)
    "Cleveland Indians": "Cleveland Guardians",
    
    # Miami Marlins (2012)
    "Florida Marlins": "Miami Marlins",
    
    # Washington Nationals (2005 relocation)
    "Montreal Expos": "Washington Nationals",
    
    # Tampa Bay Rays (2008)
    "Tampa Bay Devil Rays": "Tampa Bay Rays",
    
    # Los Angeles Angels variations
    "Anaheim Angels": "Los Angeles Angels",
    "California Angels": "Los Angeles Angels",
    "Los Angeles Angels of Anaheim": "Los Angeles Angels",
}

# Abbreviation mappings (Old -> 2025 Standard)
ABBREVIATION_MAPPINGS = {
    "OAK": "ATH",
    "CWS": "CHW",
    "FLA": "MIA",
    "MON": "WSN",
    "ANA": "LAA",
    "CAL": "LAA",
    "TBD": "TBR",
    "WAS": "WSN",
    "WSH": "WSN",
}

def standardize_team_name(name):
    """Convert a team name to the standard 2025 format"""
    if pd.isna(name):
        return name
    name = str(name).strip()
    return TEAM_NAME_MAPPINGS.get(name, name)

def standardize_abbreviation(abbr):
    """Convert an abbreviation to the standard 2025 format"""
    if pd.isna(abbr):
        return abbr
    abbr = str(abbr).strip().upper()
    return ABBREVIATION_MAPPINGS.get(abbr, abbr)

print("✓ Team name standardization functions defined")

✓ Team name standardization functions defined


### 3.2 Parse Postseason Results

Parse all postseason files to extract team wins, losses, and win percentages for each year.

In [11]:
import re

def parse_postseason_file(filepath, year):
    """
    Parse a postseason CSV file and extract team wins/losses.
    
    Returns a DataFrame with Team, Wins, Losses, Win% for that postseason.
    """
    try:
        df = pd.read_csv(filepath, header=None, names=['Series', 'Score', 'Result'])
    except:
        return None
    
    # Dictionary to track wins and losses per team
    team_stats = {}
    
    for _, row in df.iterrows():
        try:
            # Parse score (e.g., "4-1" or "1-0")
            score = str(row['Score'])
            score_match = re.match(r'(\d+)-(\d+)', score)
            if not score_match:
                continue
            winner_wins = int(score_match.group(1))
            loser_wins = int(score_match.group(2))
            
            # Parse result (e.g., "New York Yankees over Atlanta Braves")
            result = str(row['Result'])
            if ' over ' not in result:
                continue
            parts = result.split(' over ')
            winner = parts[0].strip()
            loser = parts[1].strip()
            
            # Standardize team names
            winner = standardize_team_name(winner)
            loser = standardize_team_name(loser)
            
            # Update winner stats
            if winner not in team_stats:
                team_stats[winner] = {'Wins': 0, 'Losses': 0}
            team_stats[winner]['Wins'] += winner_wins
            team_stats[winner]['Losses'] += loser_wins
            
            # Update loser stats
            if loser not in team_stats:
                team_stats[loser] = {'Wins': 0, 'Losses': 0}
            team_stats[loser]['Wins'] += loser_wins
            team_stats[loser]['Losses'] += winner_wins
            
        except Exception as e:
            continue
    
    # Convert to DataFrame
    if not team_stats:
        return None
    
    records = []
    for team, stats in team_stats.items():
        total_games = stats['Wins'] + stats['Losses']
        win_pct = stats['Wins'] / total_games if total_games > 0 else 0
        records.append({
            'Team': team,
            'Wins': stats['Wins'],
            'Losses': stats['Losses'],
            'Win%': round(win_pct, 3)
        })
    
    result_df = pd.DataFrame(records)
    result_df = result_df.sort_values('Wins', ascending=False).reset_index(drop=True)
    result_df['Year'] = year
    
    return result_df

print("✓ Postseason parsing function defined")

✓ Postseason parsing function defined


In [12]:
# Parse all postseason files and create a combined table
all_postseason_stats = []
years = range(1998, 2026)

print("Parsing postseason results for all years...\n")

for year in years:
    filepath = DATA_PATH / str(year) / f'Postseason_{year}.csv'
    if filepath.exists():
        year_df = parse_postseason_file(filepath, year)
        if year_df is not None:
            all_postseason_stats.append(year_df)
            print(f"✓ {year}: {len(year_df)} teams")
        else:
            print(f"⚠ {year}: Could not parse")
    else:
        print(f"✗ {year}: File not found")

# Combine all years into one DataFrame
postseason_all_years = pd.concat(all_postseason_stats, ignore_index=True)
print(f"\n{'='*50}")
print(f"Total records: {len(postseason_all_years)}")
print(f"Years covered: {postseason_all_years['Year'].min()} - {postseason_all_years['Year'].max()}")

Parsing postseason results for all years...

✓ 1998: 8 teams
✓ 1999: 8 teams
✓ 2000: 8 teams
✓ 2001: 8 teams
✓ 2002: 8 teams
✓ 2003: 8 teams
✓ 2004: 8 teams
✓ 2005: 8 teams
✓ 2006: 8 teams
✓ 2007: 8 teams
✓ 2008: 8 teams
✓ 2009: 8 teams
✓ 2010: 8 teams
✓ 2011: 8 teams
✓ 2012: 10 teams
✓ 2013: 10 teams
✓ 2014: 10 teams
✓ 2015: 10 teams
✓ 2016: 10 teams
✓ 2017: 10 teams
✓ 2018: 10 teams
✓ 2019: 10 teams
✓ 2020: 16 teams
✓ 2021: 10 teams
✓ 2022: 12 teams
✓ 2023: 12 teams
✓ 2024: 12 teams
✓ 2025: 12 teams

Total records: 266
Years covered: 1998 - 2025


In [13]:
# Display postseason stats for each year
print("Postseason Results by Year\n")
print("="*70)

for year in postseason_all_years['Year'].unique():
    year_data = postseason_all_years[postseason_all_years['Year'] == year].copy()
    year_data = year_data.drop(columns=['Year'])
    print(f"\n📅 {year} Postseason:")
    print(year_data.to_string(index=False))
    print("-"*50)

Postseason Results by Year


📅 1998 Postseason:
               Team  Wins  Losses  Win%
   New York Yankees    11       2 0.846
   San Diego Padres     7       7 0.500
Cleveland Guardians     5       5 0.500
     Atlanta Braves     5       4 0.556
     Boston Red Sox     1       3 0.250
     Houston Astros     1       3 0.250
      Texas Rangers     0       3 0.000
       Chicago Cubs     0       3 0.000
--------------------------------------------------

📅 1999 Postseason:
                Team  Wins  Losses  Win%
    New York Yankees    11       1 0.917
      Atlanta Braves     7       7 0.500
       New York Mets     5       5 0.500
      Boston Red Sox     4       6 0.400
 Cleveland Guardians     2       3 0.400
      Houston Astros     1       3 0.250
Arizona Diamondbacks     1       3 0.250
       Texas Rangers     0       3 0.000
--------------------------------------------------

📅 2000 Postseason:
                Team  Wins  Losses  Win%
    New York Yankees    11       5 0.688

### 3.3 Clean and Standardize All Data Files

Now let's create a comprehensive cleaning loop that standardizes team names across ALL data files (Batting, Pitching, Fielding, Salaries, WAA).

In [14]:
def clean_dataframe(df, file_type):
    """
    Clean and standardize a DataFrame by applying team name mappings.
    
    Args:
        df: pandas DataFrame to clean
        file_type: Type of file ('Batting', 'Pitching', 'Fielding', 'Salaries', 'WAA_Positions')
    
    Returns:
        Cleaned DataFrame
    """
    df_clean = df.copy()
    
    # Columns that contain team names
    team_columns = ['Tm', 'Team', 'team', 'Name']
    
    # Apply team name standardization to team columns
    for col in df_clean.columns:
        if col in team_columns:
            df_clean[col] = df_clean[col].apply(standardize_team_name)
    
    # For WAA files, the first column often has team names embedded with stats
    if file_type == 'WAA_Positions' and len(df_clean.columns) > 0:
        # Extract team names from complex strings like "Arizona Diamondbacks19.9"
        for col in df_clean.columns:
            if df_clean[col].dtype == object:
                df_clean[col] = df_clean[col].apply(lambda x: standardize_embedded_team(x) if pd.notna(x) else x)
    
    return df_clean

def standardize_embedded_team(value):
    """Handle cells that have team names embedded with numbers (e.g., 'Oakland Athletics4.5')"""
    value_str = str(value)
    for old_name, new_name in TEAM_NAME_MAPPINGS.items():
        if old_name in value_str:
            value_str = value_str.replace(old_name, new_name)
    for old_abbr, new_abbr in ABBREVIATION_MAPPINGS.items():
        # Match abbreviation at start or after comma, followed by number
        pattern = r'\b' + old_abbr + r'(?=[\d\.\-]|$)'
        value_str = re.sub(pattern, new_abbr, value_str)
    return value_str

print("✓ DataFrame cleaning functions defined")

✓ DataFrame cleaning functions defined


In [15]:
# Master cleaning loop - Load and clean all data files
file_types = ['Batting', 'Pitching', 'Fielding', 'Salaries']
years = range(1998, 2026)

# Dictionary to store all cleaned DataFrames
cleaned_data = {ft: [] for ft in file_types}
cleaning_summary = []

print("Cleaning and standardizing all data files...\n")
print("="*60)

for year in years:
    year_path = DATA_PATH / str(year)
    if not year_path.exists():
        continue
    
    year_changes = 0
    
    for file_type in file_types:
        filepath = year_path / f'{file_type}_{year}.csv'
        if filepath.exists():
            try:
                # Load the data
                df = pd.read_csv(filepath)
                original_values = df.to_string()
                
                # Clean the data
                df_clean = clean_dataframe(df, file_type)
                
                # Add year column for later merging
                df_clean['Year'] = year
                
                # Check if any changes were made
                changes = original_values != df_clean.drop(columns=['Year']).to_string()
                if changes:
                    year_changes += 1
                
                # Store cleaned DataFrame
                cleaned_data[file_type].append(df_clean)
                
            except Exception as e:
                print(f"  ⚠ Error processing {file_type}_{year}.csv: {e}")
    
    if year_changes > 0:
        print(f"✓ {year}: {year_changes} files had name standardizations")
    else:
        print(f"○ {year}: All files already standardized")

print("\n" + "="*60)
print("Cleaning complete!")

Cleaning and standardizing all data files...

✓ 1998: 3 files had name standardizations
✓ 1999: 3 files had name standardizations
✓ 2000: 3 files had name standardizations
✓ 2001: 3 files had name standardizations
✓ 2002: 3 files had name standardizations
✓ 2003: 3 files had name standardizations
✓ 2004: 3 files had name standardizations
✓ 2005: 3 files had name standardizations
✓ 2006: 3 files had name standardizations
✓ 2007: 3 files had name standardizations
✓ 2008: 3 files had name standardizations
✓ 2009: 3 files had name standardizations
✓ 2010: 3 files had name standardizations
✓ 2011: 3 files had name standardizations
✓ 2012: 3 files had name standardizations
✓ 2013: 3 files had name standardizations
✓ 2014: 3 files had name standardizations
✓ 2015: 3 files had name standardizations
✓ 2016: 3 files had name standardizations
✓ 2017: 3 files had name standardizations
✓ 2018: 3 files had name standardizations
✓ 2019: 3 files had name standardizations
✓ 2020: 3 files had name stand

In [16]:
# Combine all years into master DataFrames for each file type
batting_all = pd.concat(cleaned_data['Batting'], ignore_index=True)
pitching_all = pd.concat(cleaned_data['Pitching'], ignore_index=True)
fielding_all = pd.concat(cleaned_data['Fielding'], ignore_index=True)
salaries_all = pd.concat(cleaned_data['Salaries'], ignore_index=True)

# Remove any aggregate/league average rows (they usually have empty or NaN team names)
batting_all = batting_all[batting_all['Tm'].notna() & (batting_all['Tm'] != '')]
pitching_all = pitching_all[pitching_all['Tm'].notna() & (pitching_all['Tm'] != '')]
fielding_all = fielding_all[fielding_all['Tm'].notna() & (fielding_all['Tm'] != '')]
salaries_all = salaries_all[salaries_all['Tm'].notna() & (salaries_all['Tm'] != '')]

print("Master DataFrames Created:")
print(f"  • Batting:   {len(batting_all)} rows × {len(batting_all.columns)} columns")
print(f"  • Pitching:  {len(pitching_all)} rows × {len(pitching_all.columns)} columns")
print(f"  • Fielding:  {len(fielding_all)} rows × {len(fielding_all.columns)} columns")
print(f"  • Salaries:  {len(salaries_all)} rows × {len(salaries_all.columns)} columns")
print(f"  • Postseason: {len(postseason_all_years)} rows × {len(postseason_all_years.columns)} columns")

Master DataFrames Created:
  • Batting:   840 rows × 30 columns
  • Pitching:  840 rows × 37 columns
  • Fielding:  840 rows × 20 columns
  • Salaries:  840 rows × 3 columns
  • Postseason: 266 rows × 5 columns


### 3.4 Verify Team Name Standardization

Let's verify that all team names are now consistent across the datasets.

In [17]:
# Check unique team names across all datasets
print("Unique Team Names in Each Dataset:\n")

datasets = {
    'Batting': batting_all['Tm'].unique(),
    'Pitching': pitching_all['Tm'].unique(),
    'Fielding': fielding_all['Tm'].unique(),
    'Salaries': salaries_all['Tm'].unique(),
    'Postseason': postseason_all_years['Team'].unique()
}

for name, teams in datasets.items():
    print(f"{name}: {len(teams)} unique teams")
    
print("\n" + "="*60)
print("\nAll unique team names (sorted):")
all_teams = set()
for teams in datasets.values():
    all_teams.update(teams)
    
for team in sorted(all_teams):
    print(f"  • {team}")

Unique Team Names in Each Dataset:

Batting: 30 unique teams
Pitching: 30 unique teams
Fielding: 30 unique teams
Salaries: 30 unique teams
Postseason: 30 unique teams


All unique team names (sorted):
  • Arizona Diamondbacks
  • Athletics
  • Atlanta Braves
  • Baltimore Orioles
  • Boston Red Sox
  • Chicago Cubs
  • Chicago White Sox
  • Cincinnati Reds
  • Cleveland Guardians
  • Colorado Rockies
  • Detroit Tigers
  • Houston Astros
  • Kansas City Royals
  • Los Angeles Angels
  • Los Angeles Dodgers
  • Miami Marlins
  • Milwaukee Brewers
  • Minnesota Twins
  • New York Mets
  • New York Yankees
  • Philadelphia Phillies
  • Pittsburgh Pirates
  • San Diego Padres
  • San Francisco Giants
  • Seattle Mariners
  • St. Louis Cardinals
  • Tampa Bay Rays
  • Texas Rangers
  • Toronto Blue Jays
  • Washington Nationals


### 3.5 Filter to Playoff Teams Only

Now we filter all the data to only include teams that made the playoffs in each year. This allows us to analyze only the teams that matter for our correlation analysis.

In [33]:
def filter_to_playoff_teams(stats_df, postseason_df, team_col='Tm', year_col='Year'):
    """
    Filter a statistics DataFrame to only include teams that made the playoffs in each year.
    
    Args:
        stats_df: DataFrame with team statistics (must have team and year columns)
        postseason_df: DataFrame with postseason data (Team, Year columns)
        team_col: Name of team column in stats_df
        year_col: Name of year column
    
    Returns:
        Filtered DataFrame containing only playoff teams
    """
    # Create a set of (team, year) tuples for playoff teams
    playoff_team_years = set(zip(postseason_df['Team'], postseason_df['Year']))
    
    # Filter the stats DataFrame
    mask = stats_df.apply(lambda row: (row[team_col], row[year_col]) in playoff_team_years, axis=1)
    
    return stats_df[mask].copy()

print("✓ Playoff team filter function defined")

✓ Playoff team filter function defined


In [34]:
# Filter all statistics DataFrames to only include playoff teams
batting_playoff = filter_to_playoff_teams(batting_all, postseason_all_years)
pitching_playoff = filter_to_playoff_teams(pitching_all, postseason_all_years)
fielding_playoff = filter_to_playoff_teams(fielding_all, postseason_all_years)
salaries_playoff = filter_to_playoff_teams(salaries_all, postseason_all_years)

print("Filtered to Playoff Teams Only:")
print(f"  • Batting:  {len(batting_all)} → {len(batting_playoff)} records ({len(batting_playoff)} playoff team-years)")
print(f"  • Pitching: {len(pitching_all)} → {len(pitching_playoff)} records")
print(f"  • Fielding: {len(fielding_all)} → {len(fielding_playoff)} records")
print(f"  • Salaries: {len(salaries_all)} → {len(salaries_playoff)} records")
print(f"\nPostseason records: {len(postseason_all_years)}")

Filtered to Playoff Teams Only:
  • Batting:  840 → 266 records (266 playoff team-years)
  • Pitching: 840 → 266 records
  • Fielding: 840 → 266 records
  • Salaries: 840 → 266 records

Postseason records: 266


In [35]:
# Preview playoff teams for a sample year
sample_year = 2025
playoff_teams_2025 = postseason_all_years[postseason_all_years['Year'] == sample_year]['Team'].unique()
print(f"Playoff Teams in {sample_year}:")
for team in sorted(playoff_teams_2025):
    print(f"  • {team}")

print(f"\nBatting stats now only include these {len(playoff_teams_2025)} teams for {sample_year}:")
batting_2025_playoff = batting_playoff[batting_playoff['Year'] == sample_year]
print(batting_2025_playoff[['Tm', 'Year', 'R', 'HR', 'BA', 'OPS']].to_string(index=False))

Playoff Teams in 2025:
  • Boston Red Sox
  • Chicago Cubs
  • Cincinnati Reds
  • Cleveland Guardians
  • Detroit Tigers
  • Los Angeles Dodgers
  • Milwaukee Brewers
  • New York Yankees
  • Philadelphia Phillies
  • San Diego Padres
  • Seattle Mariners
  • Toronto Blue Jays

Batting stats now only include these 12 teams for 2025:
                   Tm  Year   R  HR    BA   OPS
       Boston Red Sox  2025 786 186 0.254 0.745
         Chicago Cubs  2025 793 223 0.249 0.751
      Cincinnati Reds  2025 716 167 0.245 0.706
  Cleveland Guardians  2025 643 168 0.226 0.670
       Detroit Tigers  2025 758 198 0.247 0.730
  Los Angeles Dodgers  2025 825 244 0.253 0.768
    Milwaukee Brewers  2025 806 166 0.258 0.736
     New York Yankees  2025 849 274 0.251 0.787
Philadelphia Phillies  2025 778 212 0.258 0.759
     San Diego Padres  2025 702 152 0.252 0.711
     Seattle Mariners  2025 766 238 0.244 0.740
    Toronto Blue Jays  2025 798 191 0.265 0.761


---

## 📊 Phase 4: Ranking Statistics

Now we'll rank each team for every statistic per year. 

**Ranking System:**
- Rank 1 = Highest value for that statistic
- Rank 30 = Lowest value for that statistic
- The rank IS the point value (1 point for highest, 30 points for lowest)

This keeps statistics separate (Batting SO ≠ Pitching SO) and allows us to track who had the most/least of each stat regardless of whether it's traditionally "good" or "bad".

In [36]:
def rank_stats_by_year(df, stat_columns, team_col='Tm', year_col='Year'):
    """
    Rank teams for each statistic within each year.
    
    Rank 1 = Highest value (gets 1 point)
    Rank 30 = Lowest value (gets 30 points)
    
    Args:
        df: DataFrame with team stats
        stat_columns: List of columns to rank
        team_col: Name of team column
        year_col: Name of year column
    
    Returns:
        DataFrame with rankings for each stat
    """
    rankings = []
    
    for year in df[year_col].unique():
        year_data = df[df[year_col] == year].copy()
        
        year_rankings = {team_col: year_data[team_col].values, year_col: year}
        
        for stat in stat_columns:
            if stat in year_data.columns:
                # Convert to numeric, coercing errors to NaN
                values = pd.to_numeric(year_data[stat], errors='coerce')
                
                # Rank descending (highest value = rank 1)
                # method='min' handles ties by giving same rank
                year_rankings[f'{stat}_Rank'] = values.rank(ascending=False, method='min').astype('Int64').values
        
        # Create DataFrame for this year
        year_df = pd.DataFrame(year_rankings)
        rankings.append(year_df)
    
    return pd.concat(rankings, ignore_index=True)

print("✓ Ranking function defined")

✓ Ranking function defined


### 4.1 Batting Statistics Rankings

In [37]:
# Define batting statistics to rank (PLAYOFF TEAMS ONLY)
batting_exclude = ['Tm', 'Year', '#Bat', 'BatAge']
batting_stat_cols = [col for col in batting_playoff.columns if col not in batting_exclude]

print(f"Batting statistics to rank ({len(batting_stat_cols)} stats):")
print(batting_stat_cols)

# Create batting rankings for PLAYOFF TEAMS ONLY
batting_rankings = rank_stats_by_year(batting_playoff, batting_stat_cols)
print(f"\n✓ Batting rankings created: {len(batting_rankings)} playoff team-year records")

Batting statistics to rank (26 stats):
['R/G', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB', 'LOB']

✓ Batting rankings created: 266 playoff team-year records


In [38]:
# Preview batting rankings
print("Batting Rankings Sample (2025):")
batting_2025 = batting_rankings[batting_rankings['Year'] == 2025]
display_cols = ['Tm', 'Year', 'R_Rank', 'H_Rank', 'HR_Rank', 'RBI_Rank', 'BA_Rank', 'OPS_Rank']
available_display = [c for c in display_cols if c in batting_2025.columns]
batting_2025[available_display].sort_values('HR_Rank')

Batting Rankings Sample (2025):


,Tm,Year,R_Rank,H_Rank,HR_Rank,RBI_Rank,BA_Rank,OPS_Rank
261,New York Yankees,2025,1,6,1,1,7,1
259,Los Angeles Dodgers,2025,2,5,2,2,5,2
264,Seattle Mariners,2025,8,10,3,8,11,7
255,Chicago Cubs,2025,5,6,4,3,8,5
262,Philadelphia Phillies,2025,7,2,5,5,2,4
258,Detroit Tigers,2025,9,9,6,9,9,9
265,Toronto Blue Jays,2025,4,1,7,3,1,3
254,Boston Red Sox,2025,6,4,8,7,4,6
257,Cleveland Guardians,2025,12,12,9,12,12,12
256,Cincinnati Reds,2025,10,11,10,10,10,11


### 4.2 Pitching Statistics Rankings

In [39]:
# Define pitching statistics to rank (PLAYOFF TEAMS ONLY)
pitching_exclude = ['Tm', 'Year', '#P', 'PAge']
pitching_stat_cols = [col for col in pitching_playoff.columns if col not in pitching_exclude]

print(f"Pitching statistics to rank ({len(pitching_stat_cols)} stats):")
print(pitching_stat_cols)

# Create pitching rankings for PLAYOFF TEAMS ONLY
pitching_rankings = rank_stats_by_year(pitching_playoff, pitching_stat_cols)
print(f"\n✓ Pitching rankings created: {len(pitching_rankings)} playoff team-year records")

Pitching statistics to rank (33 stats):
['RA/G', 'W', 'L', 'W-L%', 'ERA', 'G', 'GS', 'GF', 'CG', 'tSho', 'cSho', 'SV', 'IP', 'H', 'R', 'ER', 'HR', 'BB', 'IBB', 'SO', 'HBP', 'BK', 'WP', 'BF', 'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'LOB']

✓ Pitching rankings created: 266 playoff team-year records


In [40]:
# Preview pitching rankings
print("Pitching Rankings Sample (2025):")
pitching_2025 = pitching_rankings[pitching_rankings['Year'] == 2025]
display_cols = ['Tm', 'Year', 'W_Rank', 'ERA_Rank', 'SO_Rank', 'WHIP_Rank', 'SV_Rank']
available_display = [c for c in display_cols if c in pitching_2025.columns]
pitching_2025[available_display].sort_values('ERA_Rank')

Pitching Rankings Sample (2025):


,Tm,Year,W_Rank,ERA_Rank,SO_Rank,WHIP_Rank,SV_Rank
265,Toronto Blue Jays,2025,3,1,5,2,10
259,Los Angeles Dodgers,2025,5,2,1,4,4
258,Detroit Tigers,2025,11,2,10,5,12
261,New York Yankees,2025,3,4,3,6,8
264,Seattle Mariners,2025,7,5,6,10,8
256,Cincinnati Reds,2025,12,6,9,9,11
255,Chicago Cubs,2025,6,7,12,12,7
262,Philadelphia Phillies,2025,2,7,2,7,2
254,Boston Red Sox,2025,9,9,11,1,5
257,Cleveland Guardians,2025,10,9,8,3,2


### 4.3 Fielding Statistics Rankings

In [41]:
# Define fielding statistics to rank (PLAYOFF TEAMS ONLY)
fielding_exclude = ['Tm', 'Year', '#Fld']
fielding_stat_cols = [col for col in fielding_playoff.columns if col not in fielding_exclude]

print(f"Fielding statistics to rank ({len(fielding_stat_cols)} stats):")
print(fielding_stat_cols)

# Create fielding rankings for PLAYOFF TEAMS ONLY
fielding_rankings = rank_stats_by_year(fielding_playoff, fielding_stat_cols)
print(f"\n✓ Fielding rankings created: {len(fielding_rankings)} playoff team-year records")

Fielding statistics to rank (17 stats):
['RA/G', 'DefEff', 'G', 'GS', 'CG', 'Inn', 'Ch', 'PO', 'A', 'E', 'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'Rdrs', 'Rdrs/yr', 'Rgood']

✓ Fielding rankings created: 266 playoff team-year records


In [42]:
# Preview fielding rankings
print("Fielding Rankings Sample (2025):")
fielding_2025 = fielding_rankings[fielding_rankings['Year'] == 2025]
display_cols = ['Tm', 'Year', 'Fld%_Rank', 'E_Rank', 'DP_Rank', 'DefEff_Rank']
available_display = [c for c in display_cols if c in fielding_2025.columns]
fielding_2025[available_display].sort_values('Fld%_Rank')

Fielding Rankings Sample (2025):


,Tm,Year,Fld%_Rank,E_Rank,DP_Rank,DefEff_Rank
255,Chicago Cubs,2025,1,12,7,1
262,Philadelphia Phillies,2025,1,11,9,11
264,Seattle Mariners,2025,3,9,5,7
259,Los Angeles Dodgers,2025,3,10,11,6
260,Milwaukee Brewers,2025,5,7,3,4
263,San Diego Padres,2025,5,8,7,2
258,Detroit Tigers,2025,7,6,2,8
256,Cincinnati Reds,2025,8,4,12,3
265,Toronto Blue Jays,2025,8,5,5,9
261,New York Yankees,2025,10,3,9,5


### 4.4 Salary Rankings

In [43]:
# Define salary statistics to rank (PLAYOFF TEAMS ONLY)
salary_exclude = ['Tm', 'Year']
salary_stat_cols = [col for col in salaries_playoff.columns if col not in salary_exclude]

print(f"Salary statistics to rank ({len(salary_stat_cols)} stats):")
print(salary_stat_cols)

# Create salary rankings for PLAYOFF TEAMS ONLY
salary_rankings = rank_stats_by_year(salaries_playoff, salary_stat_cols)
print(f"\n✓ Salary rankings created: {len(salary_rankings)} playoff team-year records")

Salary statistics to rank (1 stats):
['Payroll']

✓ Salary rankings created: 266 playoff team-year records


In [44]:
# Preview salary rankings
print("Salary Rankings Sample (2025):")
salary_2025 = salary_rankings[salary_rankings['Year'] == 2025]
salary_2025.sort_values('Payroll_Rank')

Salary Rankings Sample (2025):


,Tm,Year,Payroll_Rank
254,Los Angeles Dodgers,2025,1
255,Philadelphia Phillies,2025,2
256,New York Yankees,2025,3
257,Toronto Blue Jays,2025,4
258,San Diego Padres,2025,5
259,Boston Red Sox,2025,6
260,Chicago Cubs,2025,6
261,Seattle Mariners,2025,8
262,Detroit Tigers,2025,9
263,Cincinnati Reds,2025,10


### 4.5 Rankings Summary

All ranking tables are now created and kept **separate** by category:

In [45]:
# Summary of all ranking tables (PLAYOFF TEAMS ONLY)
print("="*60)
print("RANKING TABLES SUMMARY (PLAYOFF TEAMS ONLY)")
print("="*60)
print(f"\n📊 batting_rankings:")
print(f"   • Records: {len(batting_rankings)} playoff team-years")
print(f"   • Stats ranked: {len([c for c in batting_rankings.columns if '_Rank' in c])}")
print(f"   • Years: {batting_rankings['Year'].min()} - {batting_rankings['Year'].max()}")

print(f"\n📊 pitching_rankings:")
print(f"   • Records: {len(pitching_rankings)} playoff team-years")
print(f"   • Stats ranked: {len([c for c in pitching_rankings.columns if '_Rank' in c])}")
print(f"   • Years: {pitching_rankings['Year'].min()} - {pitching_rankings['Year'].max()}")

print(f"\n📊 fielding_rankings:")
print(f"   • Records: {len(fielding_rankings)} playoff team-years")
print(f"   • Stats ranked: {len([c for c in fielding_rankings.columns if '_Rank' in c])}")
print(f"   • Years: {fielding_rankings['Year'].min()} - {fielding_rankings['Year'].max()}")

print(f"\n📊 salary_rankings:")
print(f"   • Records: {len(salary_rankings)} playoff team-years")
print(f"   • Stats ranked: {len([c for c in salary_rankings.columns if '_Rank' in c])}")
print(f"   • Years: {salary_rankings['Year'].min()} - {salary_rankings['Year'].max()}")

print(f"\n📊 postseason_all_years:")
print(f"   • Records: {len(postseason_all_years)}")
print(f"   • Columns: Team, Wins, Losses, Win%, Year")
print(f"   • Years: {postseason_all_years['Year'].min()} - {postseason_all_years['Year'].max()}")

print("\n" + "="*60)
print("✓ Only teams that made the playoffs are included")
print("✓ All tables are SEPARATE - Batting SO ≠ Pitching SO")
print("✓ Rank 1 = Highest value | Rank 30 = Lowest value")
print("="*60)

RANKING TABLES SUMMARY (PLAYOFF TEAMS ONLY)

📊 batting_rankings:
   • Records: 266 playoff team-years
   • Stats ranked: 26
   • Years: 1998 - 2025

📊 pitching_rankings:
   • Records: 266 playoff team-years
   • Stats ranked: 33
   • Years: 1998 - 2025

📊 fielding_rankings:
   • Records: 266 playoff team-years
   • Stats ranked: 17
   • Years: 1998 - 2025

📊 salary_rankings:
   • Records: 266 playoff team-years
   • Stats ranked: 1
   • Years: 1998 - 2025

📊 postseason_all_years:
   • Records: 266
   • Columns: Team, Wins, Losses, Win%, Year
   • Years: 1998 - 2025

✓ Only teams that made the playoffs are included
✓ All tables are SEPARATE - Batting SO ≠ Pitching SO
✓ Rank 1 = Highest value | Rank 30 = Lowest value


---

## ✅ Data Ready for Analysis

**Separate Ranking Tables Created:**

| Table | Category | Stats Ranked | Purpose |
|-------|----------|--------------|---------|
| `batting_rankings` | Batting | R, H, HR, RBI, BA, OBP, SLG, OPS, SO, etc. | Team offensive rankings |
| `pitching_rankings` | Pitching | W, L, ERA, SO, BB, WHIP, SV, etc. | Team pitching rankings |
| `fielding_rankings` | Fielding | Fld%, E, DP, DefEff, etc. | Team defense rankings |
| `salary_rankings` | Salary | Payroll | Team spending rankings |
| `postseason_all_years` | Playoffs | Wins, Losses, Win% | Playoff performance |

**Key Points:**
- Each category's statistics are kept in **separate tables**
- Batting SO (strikeouts by batters) ≠ Pitching SO (strikeouts by pitchers)
- Rank 1 = Highest value for that stat
- Rank 30 = Lowest value for that stat

---

## 🚀 Next Steps

1. **Link rankings to postseason performance** - Join playoff teams with their regular season rankings
2. **Calculate correlations** - Which stat rankings correlate with playoff wins?
3. **Build point accumulation table** - Track which stats predict playoff success

---

## 🏆 Phase 5: Extreme Rankings Analysis

### What makes playoff teams special?

Now for the fun part! Let's look at every playoff team across all years and find out what they were **THE BEST** (Rank 1) or **THE WORST** (Rank 30) at among all playoff teams.

**Why are we doing this?**
- By identifying which stats playoff teams rank #1 or #30 in, we can start to see patterns
- Teams that win games in the playoffs might share common strengths (or weaknesses!)
- This gives us a "fingerprint" of what each playoff team was known for that year
- Later, we can correlate these extreme rankings with actual playoff wins to find what really matters

In [52]:
def get_extreme_rankings(team, year, batting_df, pitching_df, fielding_df, salary_df):
    """
    Find all rank 1 (best) and rank 30 (worst) stats for a given team in a given year.
    Returns a dict with 'best' and 'worst' lists containing formatted strings.
    
    Note: Ranking DataFrames use 'Tm' as the team column.
    """
    extremes = {'best': [], 'worst': []}
    
    # Check batting stats
    bat_row = batting_df[(batting_df['Tm'] == team) & (batting_df['Year'] == year)]
    if not bat_row.empty:
        rank_cols = [c for c in bat_row.columns if c.endswith('_Rank')]
        for col in rank_cols:
            val = bat_row[col].values[0]
            if pd.notna(val):  # Check for NA/NaN values
                stat_name = col.replace('_Rank', '')
                if val == 1:
                    extremes['best'].append(f"🏅 BATTING {stat_name}")
                elif val == max(batting_df[(batting_df['Year'] == year)][col].dropna()):
                    # Use max rank for "worst" instead of hardcoded 30 (number of teams varies)
                    extremes['worst'].append(f"📉 BATTING {stat_name}")
    
    # Check pitching stats
    pitch_row = pitching_df[(pitching_df['Tm'] == team) & (pitching_df['Year'] == year)]
    if not pitch_row.empty:
        rank_cols = [c for c in pitch_row.columns if c.endswith('_Rank')]
        for col in rank_cols:
            val = pitch_row[col].values[0]
            if pd.notna(val):  # Check for NA/NaN values
                stat_name = col.replace('_Rank', '')
                if val == 1:
                    extremes['best'].append(f"⚾ PITCHING {stat_name}")
                elif val == max(pitching_df[(pitching_df['Year'] == year)][col].dropna()):
                    extremes['worst'].append(f"📉 PITCHING {stat_name}")
    
    # Check fielding stats
    field_row = fielding_df[(fielding_df['Tm'] == team) & (fielding_df['Year'] == year)]
    if not field_row.empty:
        rank_cols = [c for c in field_row.columns if c.endswith('_Rank')]
        for col in rank_cols:
            val = field_row[col].values[0]
            if pd.notna(val):  # Check for NA/NaN values
                stat_name = col.replace('_Rank', '')
                if val == 1:
                    extremes['best'].append(f"🧤 FIELDING {stat_name}")
                elif val == max(fielding_df[(fielding_df['Year'] == year)][col].dropna()):
                    extremes['worst'].append(f"📉 FIELDING {stat_name}")
    
    # Check salary stats
    sal_row = salary_df[(salary_df['Tm'] == team) & (salary_df['Year'] == year)]
    if not sal_row.empty:
        rank_cols = [c for c in sal_row.columns if c.endswith('_Rank')]
        for col in rank_cols:
            val = sal_row[col].values[0]
            if pd.notna(val):  # Check for NA/NaN values
                stat_name = col.replace('_Rank', '')
                if val == 1:
                    extremes['best'].append(f"💰 SALARY {stat_name}")
                elif val == max(salary_df[(salary_df['Year'] == year)][col].dropna()):
                    extremes['worst'].append(f"📉 SALARY {stat_name}")
    
    return extremes

print("✓ Extreme rankings function defined")

✓ Extreme rankings function defined


In [62]:
# Compact view - World Series winners (most playoff wins) for each year
import textwrap

# Stats that appear in multiple categories and need prefixes
AMBIGUOUS_STATS = {'SO', 'BB', 'HR', 'H', 'R', 'W', 'L', 'HBP', 'IBB', 'BK', 'WP', 'CG', 'SV', 'IP', 'ER', 'E', 'DP', 'PO', 'A'}

# Stats to exclude:
# - Games-related: G, GS, GF (all teams play same number)
# - Per-game redundant: R/G ranks same as R since all teams play 162 games
# - Other redundant pairs where one is derived from the other with same ranking
EXCLUDE_STATS = {
    'G', 'GS', 'GF',           # Games stats (meaningless - all teams play 162)
    'R/G',                      # Redundant with R (same ranking when equal games)
    'PA/G',                     # Redundant with PA
    'AB/G',                     # Redundant with AB  
    'H/G',                      # Redundant with H
    'HR/G',                     # Redundant with HR
    'BB/G',                     # Redundant with BB
    'SO/G',                     # Redundant with SO
    'IP/G',                     # Redundant with IP
    'SO9',                      # Redundant with SO (per 9 IP is essentially same ranking)
    'BB9',                      # Redundant with BB
    'H9',                       # Redundant with H allowed
}

def format_stat_with_category(stat_str):
    """Extract category and stat name, add prefix if ambiguous, exclude useless stats."""
    parts = stat_str.split()
    if len(parts) >= 2:
        category = parts[1]  # BATTING, PITCHING, FIELDING, SALARY
        stat_name = parts[-1]
        
        # Skip excluded stats
        if stat_name in EXCLUDE_STATS:
            return None
        
        # Add category prefix for ambiguous stats
        if stat_name in AMBIGUOUS_STATS:
            prefix = category[0]  # B, P, F, S
            return f"{prefix}-{stat_name}"
        return stat_name
    return stat_str

print("=" * 80)
print("WORLD SERIES CHAMPIONS - WHAT MADE THEM SPECIAL?")
print("=" * 80)
print()

years_to_analyze = sorted(postseason_all_years['Year'].unique())

for year in years_to_analyze:
    # Get the team with the most wins (World Series winner)
    year_postseason = postseason_all_years[postseason_all_years['Year'] == year]
    champion = year_postseason.loc[year_postseason['Wins'].idxmax()]
    
    team = champion['Team']
    wins = champion['Wins']
    losses = champion['Losses']
    
    extremes = get_extreme_rankings(team, year, batting_rankings, pitching_rankings, 
                                    fielding_rankings, salary_rankings)
    
    # Format stats with category prefix for ambiguous ones, exclude useless ones
    rank1_stats = [format_stat_with_category(s) for s in extremes['best']]
    rank1_stats = [s for s in rank1_stats if s]  # Remove None values
    rank30_stats = [format_stat_with_category(s) for s in extremes['worst']]
    rank30_stats = [s for s in rank30_stats if s]  # Remove None values
    
    # Format as strings
    rank1_str = ", ".join(rank1_stats) if rank1_stats else "None"
    rank30_str = ", ".join(rank30_stats) if rank30_stats else "None"
    
    print(f"{year} {team} ({wins}W-{losses}L)")
    
    # Wrap text to fit within 75 characters
    wrapped_rank1 = textwrap.fill(f"Rank 1: {rank1_str}", width=75, subsequent_indent="         ")
    wrapped_rank30 = textwrap.fill(f"Rank 30: {rank30_str}", width=75, subsequent_indent="          ")
    
    print(f"   {wrapped_rank1}")
    print(f"   {wrapped_rank30}")
    print()

WORLD SERIES CHAMPIONS - WHAT MADE THEM SPECIAL?

1998 New York Yankees (11W-2L)
   Rank 1: PA, B-R, RBI, CS, B-BB, OBP, OPS, OPS+, SF, P-W, W-L%, P-HBP,
         DefEff, F-CG, Payroll
   Rank 30: B-SO, P-L, P-IBB

1999 New York Yankees (11W-1L)
   Rank 1: B-HBP, P-SV, P-HBP, LOB, Payroll
   Rank 30: SH, Rtot, Rtot/yr

2000 New York Yankees (11W-5L)
   Rank 1: P-L, ERA, P-ER, P-WP, Payroll
   Rank 30: SH, P-W, W-L%, tSho, cSho, P-IP, P-IBB, Inn, Ch, F-PO, F-A

2001 Arizona Diamondbacks (11W-6L)
   Rank 1: B-IBB, P-SO, SO/W, Fld%
   Rank 30: GDP, SF, P-SV, LOB

2002 Los Angeles Angels (11W-5L)
   Rank 1: AB, B-H, SB, BA, B-HBP, SF, FIP, DefEff, Rtot, Rtot/yr
   Rank 30: B-HR, B-BB, B-SO, GDP, cSho, P-IBB

2003 Miami Marlins (11W-6L)
   Rank 1: SB, CS, SH, P-BK, Fld%
   Rank 30: PA, GDP, LOB, P-SV, P-HR, P-HBP, HR9, F-E, Rdrs/yr, Payroll

2004 Boston Red Sox (11W-3L)
   Rank 1: PA, AB, B-R, B-H, 2B, RBI, B-SO, BA, OBP, SLG, OPS, TB, LOB, P-CG,
         P-HBP, F-E
   Rank 30: SB, CS, SH, 

In [63]:
# Analyze most common Rank 1 and Rank 30 stats among World Series Champions
from collections import Counter

all_rank1_stats = []
all_rank30_stats = []

for year in years_to_analyze:
    year_postseason = postseason_all_years[postseason_all_years['Year'] == year]
    champion = year_postseason.loc[year_postseason['Wins'].idxmax()]
    team = champion['Team']
    
    extremes = get_extreme_rankings(team, year, batting_rankings, pitching_rankings, 
                                    fielding_rankings, salary_rankings)
    
    # Format stats with category prefix for ambiguous ones, exclude useless ones
    rank1_stats = [format_stat_with_category(s) for s in extremes['best']]
    rank1_stats = [s for s in rank1_stats if s]
    rank30_stats = [format_stat_with_category(s) for s in extremes['worst']]
    rank30_stats = [s for s in rank30_stats if s]
    
    all_rank1_stats.extend(rank1_stats)
    all_rank30_stats.extend(rank30_stats)

# Count occurrences
rank1_counts = Counter(all_rank1_stats)
rank30_counts = Counter(all_rank30_stats)

# Stat explanations
STAT_EXPLANATIONS = {
    # Batting stats
    'R/G': ('Runs per Game', 'Scoring runs is the fundamental way to win. Champions average more runs per game than other playoff teams.'),
    'B-R': ('Batting Runs', 'Total runs scored by the offense. More runs = more wins. Simple as that.'),
    'RBI': ('Runs Batted In', 'Clutch hitting that drives in teammates. Champions get hits when it matters.'),
    'OBP': ('On-Base Percentage', 'Getting on base creates scoring opportunities. High OBP = more chances to score.'),
    'SLG': ('Slugging Percentage', 'Power hitting. Extra-base hits drive in more runs and change games.'),
    'OPS': ('On-Base Plus Slugging', 'The best single batting stat - combines getting on base AND hitting for power.'),
    'OPS+': ('OPS+ (adjusted)', 'OPS adjusted for park/league. 100 is average, higher is better. Shows true offensive quality.'),
    'BA': ('Batting Average', 'Classic hitting stat. Getting hits keeps rallies alive.'),
    'TB': ('Total Bases', 'Raw offensive production. More bases = more damage per at-bat.'),
    'LOB': ('Left on Base', 'Runners stranded. High LOB means lots of baserunners (good!) but also missed opportunities.'),
    'B-HR': ('Home Runs (Batting)', 'The ultimate run producer. One swing can change a game or series.'),
    'B-SO': ('Strikeouts (Batting)', 'High strikeouts mean less contact. Champions often have power hitters who strike out.'),
    'B-BB': ('Walks (Batting)', 'Plate discipline. Drawing walks extends innings and wears out pitchers.'),
    
    # Pitching stats
    'P-W': ('Pitching Wins', 'Team wins attributed to pitching. Good pitching wins championships.'),
    'W-L%': ('Win-Loss Percentage', 'The most direct measure of success. Champions win more than they lose.'),
    'P-L': ('Pitching Losses', 'Fewer losses = more consistent pitching. Champions avoid bad outings.'),
    'ERA': ('Earned Run Average', 'Runs allowed per 9 innings. Lower ERA = better pitching.'),
    'ERA+': ('ERA+ (adjusted)', 'ERA adjusted for park/league. 100 is average, higher is better.'),
    'WHIP': ('Walks + Hits per IP', 'Baserunners allowed. Lower WHIP = fewer scoring threats.'),
    'P-SO': ('Strikeouts (Pitching)', 'Dominant pitching. Strikeouts eliminate baserunners completely.'),
    'P-BB': ('Walks (Pitching)', 'Free passes hurt. Fewer walks = better control.'),
    'P-SV': ('Saves', 'Closing out games. Low saves can mean blowout wins (no save situation needed).'),
    'P-IBB': ('Intentional Walks (Pitching)', 'Strategic walks. Low IBB may mean pitchers attack hitters confidently.'),
    'cSho': ('Complete Shutouts', 'Rare in modern baseball. Low cSho is normal with bullpen usage.'),
    'P-HR': ('Home Runs Allowed', 'Giving up dingers. Fewer HRs allowed = better pitching.'),
    'HR9': ('HR per 9 Innings', 'Home run rate. Lower is better for pitchers.'),
    'FIP': ('Fielding Indep. Pitching', 'What pitcher controls (K, BB, HR). Lower = better true pitching.'),
    'LOB': ('Left on Base (Pitching)', 'Stranding runners. High LOB% means escaping jams.'),
    
    # Fielding stats
    'Fld%': ('Fielding Percentage', 'Plays made without errors. Higher = cleaner defense.'),
    'F-E': ('Errors', 'Mistakes in the field. Fewer errors = more reliable defense.'),
    'DefEff': ('Defensive Efficiency', 'Balls in play converted to outs. Higher = better range.'),
    'F-DP': ('Double Plays', 'Getting two outs at once. Great for escaping jams.'),
    'Rtot': ('Total Runs Saved', 'Runs prevented by defense. Higher = more valuable defense.'),
    'Rtot/yr': ('Runs Saved per Year', 'Defensive value normalized. Shows true defensive impact.'),
    
    # Salary
    'Payroll': ('Team Payroll', 'Money spent on players. Higher payroll often means better talent.'),
}

print("=" * 80)
print("TOP 5 STATS WORLD SERIES CHAMPIONS RANKED #1 IN (1998-2025)")
print("=" * 80)
print()
for stat, count in rank1_counts.most_common(5):
    pct = count / len(years_to_analyze) * 100
    if stat in STAT_EXPLANATIONS:
        name, why = STAT_EXPLANATIONS[stat]
        print(f"  {stat} ({name}): {count} champions ({pct:.0f}%)")
        print(f"      → {why}")
    else:
        print(f"  {stat}: {count} champions ({pct:.0f}%)")
    print()

print("=" * 80)
print("TOP 5 STATS WORLD SERIES CHAMPIONS RANKED LAST IN (1998-2025)")
print("=" * 80)
print()
for stat, count in rank30_counts.most_common(5):
    pct = count / len(years_to_analyze) * 100
    if stat in STAT_EXPLANATIONS:
        name, why = STAT_EXPLANATIONS[stat]
        print(f"  {stat} ({name}): {count} champions ({pct:.0f}%)")
        # Add context for why being LAST might actually be okay
        if stat == 'P-L':
            print(f"      → Being LAST in losses is GOOD! Champions lose the fewest games.")
        elif stat == 'cSho':
            print(f"      → Complete shutouts are rare in modern baseball. This doesn't hurt.")
        elif stat == 'P-SV':
            print(f"      → Low saves often means blowout wins where closers aren't needed!")
        elif stat == 'P-IBB':
            print(f"      → Few intentional walks = pitchers confidently attack hitters.")
        else:
            print(f"      → {why}")
    else:
        print(f"  {stat}: {count} champions ({pct:.0f}%)")
    print()

print("=" * 80)
print("KEY INSIGHT:")
print("Champions consistently lead in SCORING (R/G, B-R) and WINNING (P-W, W-L%).")
print("Being 'worst' in losses (P-L) is actually great - it means they WIN more!")
print("Stats like saves and shutouts being low shows modern baseball strategy.")
print("=" * 80)

TOP 5 STATS WORLD SERIES CHAMPIONS RANKED #1 IN (1998-2025)

  B-R (Batting Runs): 8 champions (29%)
      → Total runs scored by the offense. More runs = more wins. Simple as that.

  P-W (Pitching Wins): 8 champions (29%)
      → Team wins attributed to pitching. Good pitching wins championships.

  W-L% (Win-Loss Percentage): 8 champions (29%)
      → The most direct measure of success. Champions win more than they lose.

  LOB (Left on Base (Pitching)): 8 champions (29%)
      → Stranding runners. High LOB% means escaping jams.

  RBI (Runs Batted In): 7 champions (25%)
      → Clutch hitting that drives in teammates. Champions get hits when it matters.

TOP 5 STATS WORLD SERIES CHAMPIONS RANKED LAST IN (1998-2025)

  P-L (Pitching Losses): 8 champions (29%)
      → Being LAST in losses is GOOD! Champions lose the fewest games.

  LOB (Left on Base (Pitching)): 8 champions (29%)
      → Stranding runners. High LOB% means escaping jams.

  P-IBB (Intentional Walks (Pitching)): 7 cha